In [1]:
import pymssql
import re
from collections import OrderedDict
from pandas import Series,DataFrame
from tqdm import tqdm_notebook
import pandas as pd
import copy

import numpy as np

import pickle

import tensorflow as tf

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Convolution2D,Activation,MaxPooling2D,Concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.models import load_model
from sklearn.model_selection import train_test_split

import os

Using TensorFlow backend.


In [2]:
with open('shoda4000.pickle','rb') as fr: #피클 파일 불러와서 합치기
    result3 = pickle.load(fr)

In [4]:
result3.LABEL.value_counts()

1    3681
2    3670
3    3377
6    3364
4    2784
5    2685
0    2313
Name: LABEL, dtype: int64

In [5]:
#다른 크기 이미지 제거후 인덱스 섞어주기
for index in result3.index:
    if result3.loc[index]['pl_imgurl'].shape != (28,28,3): 
        result3.drop(index, inplace = True)
result3 = pd.concat([result3],ignore_index=True) 
result3 = result3.reindex(np.random.permutation(result3.index)) 

In [6]:
#원핫벡터 정답 label 만들기
from keras.utils.np_utils import to_categorical
labels = to_categorical(result3['LABEL'], num_classes =8) 
print(labels[:10])

[[ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]]


In [7]:
#트레이닝 테스트 셋 분할
data_train_image,data_test_image,labels_train_image,labels_test_image = train_test_split(np.array(result3['pl_imgurl'].tolist()),labels, test_size = 0.25, random_state=42)

In [8]:
#cnn모델 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')


model = Sequential()
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu', #3x3필터 16개를 적용, same:입력이미지와 출력 이미지크기같음
                        input_shape=data_train_image.shape[1:]))

model.add(MaxPooling2D(pool_size=(2, 2))) #2,2 중 가장 높은값을 가져옴 - 사소한 특징 무시
model.add(Dropout(0.25))
  
model.add(Convolution2D(64, 3, 3,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
  
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8,activation = 'softmax'))
  
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(data_train_image, labels_train_image, batch_size=200,validation_split = 0.2, nb_epoch=100,callbacks=[early_stopping])

# model.save('Gersang.h5')

/home/ml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 3..., padding="same")`
  
/home/ml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/home/ml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/home/ml/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 13078 samples, validate on 3270 samples
Epoch 1/100


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node loss/mul/_129}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1020_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [30]:
#이미지 샘플링 할 경우
# from keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping()
# keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

# from keras.preprocessing.image import ImageDataGenerator
# model = Sequential()
# model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu', #3x3필터 16개를 적용, same:입력이미지와 출력 이미지크기같음
#                         input_shape=data_train_image.shape[1:]))
# model.add(MaxPooling2D(pool_size=(2, 2))) #2,2 중 가장 높은값을 가져옴 - 사소한 특징 무시
# model.add(Dropout(0.25))
  
# model.add(Convolution2D(64, 3, 3,  activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
 
# model.add(Convolution2D(64, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
  
# model.add(Flatten())
# model.add(Dense(128, activation = 'relu')) # 256
# model.add(Dropout(0.5))
# model.add(Dense(8,activation = 'softmax'))
  
# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# #이미지 겹치기로 늘려주면 좋음
# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest') #try constant

# datagen.fit(data_train_image)

# model.fit_generator(datagen.flow(data_train_image,labels_train_image,batch_size =3),validation_data=(data_test_image, labels_test_image),verbose = 2, epochs=100,steps_per_epoch = len(data_test_image) / 32,callbacks=[early_stopping])

# model.save('Gersang.h5')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 3..., padding="same")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  app.launch_new_instance()


Epoch 1/100
 - 9s - loss: 2.0648 - acc: 0.1520 - val_loss: 2.0379 - val_acc: 0.1635
Epoch 2/100
 - 8s - loss: 2.0466 - acc: 0.1529 - val_loss: 2.0406 - val_acc: 0.1371


In [32]:
accr = model.evaluate(data_test_image,labels_test_image)

11617/11617 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 251us/step


In [33]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 2.041
  Accuracy: 0.137


In [29]:
# inception v3 trans learning
import tensorflow as tf
import keras.applications
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
base_model = InceptionV3(weights='imagenet', include_top=False)

CLASSES = 8
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
prev_model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model = Sequential()
model.add(prev_model)

model.add(Dense(256, activation = 'relu'))

model.add(Dense(8,activation = 'softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
from keras.applications.inception_v3 import preprocess_input
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
datagen.fit(data_train_image)

EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS = 64
MODEL_FILE = 'filename.model'
history = model.fit_generator(
    datagen.flow(data_train_image,labels_train_image,batch_size =3),
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(data_test_image, labels_test_image),
    validation_steps=VALIDATION_STEPS)
  

# model.save(MODEL_FILE)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 27879 samples, validate on 6970 samples
Epoch 1/100
22900/27879 [=======================>......] - ETA: 12:31 - loss: nan - acc: 0.140 - ETA: 6:26 - loss: nan - acc: 0.155 - ETA: 4:22 - loss: nan - acc: 0.14 - ETA: 3:20 - loss: nan - acc: 0.15 - ETA: 2:42 - loss: nan - acc: 0.14 - ETA: 2:17 - loss: nan - acc: 0.13 - ETA: 2:00 - loss: nan - acc: 0.13 - ETA: 1:46 - loss: nan - acc: 0.13 - ETA: 1:36 - loss: nan - acc: 0.13 - ETA: 1:27 - loss: nan - acc: 0.12 - ETA: 1:20 - loss: nan - acc: 0.12 - ETA: 1:15 - loss: nan - acc: 0.11 - ETA: 1:10 - loss: nan - acc: 0.12 - ETA: 1:06 - loss: nan - acc: 0.11 - ETA: 1:03 - loss: nan - acc: 0.11 - ETA: 1:00 - loss: nan - acc: 0.11 - ETA: 57s - loss: nan - acc: 0.1194 - ETA: 54s - loss: nan - acc: 0.122 - ETA: 52s - loss: nan - acc: 0.122 - ETA: 50s - loss: nan - acc: 0.124 - ETA: 48s - loss: nan - acc: 0.124 - ETA: 46s - loss: nan - acc: 0.124 - ETA: 45s - loss: nan - acc: 0.126 - ETA: 44s - loss: nan - acc: 0.127 - ETA: 43s - loss: nan - a

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:543: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


NameError: name 'MODEL_FILE' is not defined